
<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg" style="vertical-align: top; padding-top: 2px" width="08%"/></center><p></p><h1><font color="#306998"></font><center>Value-at-Risk</center></h1>
<center><b>Kannan Singaravelu</b></center>
<center><font size="3">March, 2022</font></center><hr/>



<h1 id="Value-at-Risk">Value-at-Risk<a class="anchor-link" href="#Value-at-Risk">¶</a></h1><p>Value at Risk - <strong>VaR</strong> - is one of the most important metrics that is used to measures the risk associated with a financial position or a portfolio of financial instruments. VaR can be defined as the maximum loss with a confidence level over a predetermined period. Let's say that the 1-day 95% VaR of a portfolio is $\$100$. This means that 95% of the time, it is expected that - under normal market conditions - we will not lose more than $100 by holding our portfolio over one day.</p>
<p>Three approaches that are commonly used in the industry are</p>
<ul>
<li><strong>Parametric</strong></li>
<li><strong>Historical</strong> </li>
<li><strong>Monte Carlo</strong></li>
</ul>



<h2 id="Import-Libraries">Import Libraries<a class="anchor-link" href="#Import-Libraries">¶</a></h2>


In [ ]:

# Data manipulation
import pandas as pd
import numpy as np
from numpy.linalg import multi_dot

from scipy.stats import norm
from tabulate import tabulate

# Import matplotlib for visualization
import matplotlib
import matplotlib.pyplot as plt

# Plot settings
plt.style.use('dark_background')
matplotlib.rcParams['figure.figsize'] = [24.0, 8.0]
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['lines.linewidth'] = 2.0
matplotlib.rcParams['grid.color'] = 'black'

from helper import plot_var




<h2 id="Retrieve-Data">Retrieve Data<a class="anchor-link" href="#Retrieve-Data">¶</a></h2><p>We will use the FAANG stocks as before to build for calculation of VaR</p>


In [ ]:

# Load locally stored data
df = pd.read_csv('data/nasdaqstocks.csv', index_col=0, parse_dates=True)

# Check values 
df




<h2 id="Calculate-Returns">Calculate Returns<a class="anchor-link" href="#Calculate-Returns">¶</a></h2>


In [ ]:

# Calculate daily returns
returns = df.pct_change().dropna()
returns.head()



In [ ]:

# Visualize daily returns
fig, ax = plt.subplots(1,len(df.columns), sharey=True)
label, color = df.columns, ['green', 'red', 'cornflowerblue', 'orange', 'white']

for i in range(len(df.columns)):
    ax[i].plot(returns.iloc[:,i], label=label[i], color=color[i])
    ax[i].axhline(y=0, color='k', linestyle='--')
    ax[i].legend(loc=4)

fig.suptitle('Daily Returns');




<h2 id="Parametric-VaR">Parametric VaR<a class="anchor-link" href="#Parametric-VaR">¶</a></h2><p>The Variance-covariance is a parametric method which assumes (almost always) that the returns are normally distributed. In this method, we first calculate the mean and standard deviation of the returns to derive the risk metric. Based on the assumption of normality, we can generalise,
<br/><br/>
$ VaR = position * (\mu - z * \sigma) $</p>
<table>
<thead><tr>
<th style="text-align:left">Confidence Level</th>
<th style="text-align:left">Value At Risk     </th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left"><code>90%</code></td>
<td style="text-align:left">$\mu$ - $1.29$ * $\sigma$ </td>
</tr>
<tr>
<td style="text-align:left"><code>95%</code></td>
<td style="text-align:left">$\mu$ - $1.64$ * $\sigma$             </td>
</tr>
<tr>
<td style="text-align:left"><code>99%</code></td>
<td style="text-align:left">$\mu$ - $2.33$ * $\sigma$             </td>
</tr>
</tbody>
</table>
<p>where, $\mu$ is the return, $\sigma$ is the volatility and $z$ is the number of standard deviation from the mean.</p>


In [ ]:

# Visualize VaR at 95% confidence level
plot_var()



In [ ]:

# Calculate mean and standard deviation 
mean = np.mean(returns['AMD'])
stdev = np.std(returns['AMD'])

# Calculate VaR at difference confidence level
VaR_90 = norm.ppf(1-0.90,mean,stdev)
VaR_95 = norm.ppf(1-0.95,mean,stdev) 
VaR_99 = norm.ppf(1-0.99,mean,stdev)



In [ ]:

# number of stdev from the mean
norm.ppf(0.05)



In [ ]:

# Ouput results in tabular format
table = [['90%', VaR_90],['95%', VaR_95],['99%', VaR_99] ]
header = ['Confidence Level', 'Value At Risk']
print(tabulate(table,headers=header))




<p>Lets now define a VaR function so that we can use it calculate it for individual stocks</p>



<p><strong>UDF : User Defined Function</strong></p>


In [ ]:

# VaR function
def VaR(dataframe):
    var = pd.DataFrame()

    for i in [90, 95, 99]:
        for j in range(len(df.columns)):
            var.loc[i, j] = 100 * norm.ppf(1-i/100, dataframe.iloc[:,j].mean(), dataframe.iloc[:,j].std())

    var.columns = df.columns
    return var



In [ ]:

# VaR for stocks
VaR(returns)




<p>Now, let's assume that we have 1,000 shares of AMD's stock on Dec 30, 2021. What is the maximum loss next day with a confidence level of 99%?</p>


In [ ]:

num_of_shares = 1000
price = df['AMD'].iloc[-1]
position = num_of_shares * price 

amd_var = position * VaR_99

print(f'AMD Holding Value: {position}')
print(f'AMD VaR at 99% confidence level is: {amd_var}')




<p>VaR can also be calculated using the above formula at 99% confidence level.</p>


In [ ]:

# VaR calculation by appling direct formulae
position * (mean + norm.ppf(1-0.99) * stdev)         # mean-2.33*stdev




<h2 id="Historical-VaR">Historical VaR<a class="anchor-link" href="#Historical-VaR">¶</a></h2><p>Asset returns do not necessarily follow a normal distribution. An alternative is to use sorted returns to evaluate a VaR. This method uses historical data where returns are sorted in ascending order to calculate maximum possible loss for a given confidence level.</p>


In [ ]:

# Use quantile function for Historical VaR
hVaR_90 = returns['AMD'].quantile(0.10)
hVaR_95 = returns['AMD'].quantile(0.05)
hVaR_99 = returns['AMD'].quantile(0.01)



In [ ]:

htable = [['90%', hVaR_90],['95%', hVaR_95],['99%', hVaR_99]]
print(tabulate(htable,headers=header))




<h2 id="MonteCarlo-VaR">MonteCarlo VaR<a class="anchor-link" href="#MonteCarlo-VaR">¶</a></h2><p>The Monte Carlo simulation approach has a number of similarities to historical simulation. It allows us to use actual historical distributions rather than having to assume normal returns. As returns are assumed to follow a normal distribution, we could generate <em><code>n</code></em> simulated returns with the same mean and standard deviation (derived from the daily returns) and then sorted in ascending order to calculate maximum possible loss for a given confidence level.</p>


In [ ]:

# Set seed for reproducibility
np.random.seed(42)

# Number of simulations
n_sims = 5000

# Simulate returns and sort
sim_returns = np.random.normal(mean, stdev, n_sims)

# Use percentile function for MCVaR
MCVaR_90 = np.percentile(sim_returns,10)
MCVaR_95 = np.percentile(sim_returns, 5)
MCVaR_99 = np.percentile(sim_returns,1)



In [ ]:

mctable = [['90%', MCVaR_90],['95%', MCVaR_95],['99%', MCVaR_99]]
print(tabulate(mctable,headers=header))




<h2 id="Scaling-VaR">Scaling VaR<a class="anchor-link" href="#Scaling-VaR">¶</a></h2><p>Now, let's calculate VaR over a 5-day period. To scale it, multiply by square root of time.
<br/><br/>
$$ VaR = position * (\mu - z * \sigma) * \sqrt{T}$$</p>
<p>where, $T$ is the horizon or forecast period.</p>


In [ ]:

forecast_days = 5
f_VaR_90 = VaR_90*np.sqrt(forecast_days)
f_VaR_95 = VaR_95*np.sqrt(forecast_days)
f_VaR_99 = VaR_99*np.sqrt(forecast_days)



In [ ]:

ftable = [['90%', f_VaR_90],['95%', f_VaR_95],['99%', f_VaR_99] ]
fheader = ['Confidence Level', '5-Day Forecast Value At Risk']
print(tabulate(ftable,headers=fheader))




<p>Let's now calculate AMD VaR over a 5-day period with a confidence level of 99%</p>


In [ ]:

amd_var_5days = position * f_VaR_99

print(f'AMD Holding Value: {position}')
print(f'AMD VaR at 99% confidence level is: {amd_var_5days}')



In [ ]:

# Scaled VaR over different time horizon
# plt.figure()
plt.plot(range(100),[-100*VaR_95*np.sqrt(x) for x in range(100)], color='orange')
plt.xlabel('Horizon')
plt.ylabel('Var 95 (%)')
plt.title('VaR_95 Scaled by Time');




<h2 id="Expected-Short-Fall">Expected Short Fall<a class="anchor-link" href="#Expected-Short-Fall">¶</a></h2><p>VaR is a reasonable measure of risk if assumption of normality holds. Else, we might underestimate the risk if we observe a fat tail or overestimate the risk if tail is thinner. Expected shortfall or Conditional Value at Risk - <strong>CVaR</strong> - is an estimate of expected shortfall sustained in the worst 1 - x% of scenarios. It is defined as the average loss based on the returns that are lower than the VaR threshold. Assume that we have <em><code>n</code></em> return observations, then the expected shortfall is
<br/><br/>
$$ CVaR = \frac 1 n * \sum_{i=1}^{n} R_i[R \leq hVaR_{cl}]$$</p>
<p>where, $R$ is returns, $hVaR$ is historical VaR and $cl$ is the confidence level.</p>


In [ ]:

# Calculate CVar
CVaR_90 = returns['AMD'][returns['AMD']<=hVaR_90].mean()
CVaR_95 = returns['AMD'][returns['AMD']<=hVaR_95].mean()
CVaR_99 = returns['AMD'][returns['AMD']<=hVaR_99].mean()



In [ ]:

ctable = [['90%', CVaR_90],['95%', CVaR_95],['99%', CVaR_99] ]
cheader = ['Confidence Level', 'Conditional Value At Risk']
print(tabulate(ctable,headers=cheader))




<h2 id="Portfolio-VaR">Portfolio VaR<a class="anchor-link" href="#Portfolio-VaR">¶</a></h2><p>If we know the returns and volatilities of all the assets in the portfolio, we can calculate the VaR for the whole portfolio. We will now derive VaR of an equal weighted portfolio of FAANG stocks.</p>



<h3 id="Assign-Weights">Assign Weights<a class="anchor-link" href="#Assign-Weights">¶</a></h3>


In [ ]:

# Weights from Minimum Variance Portfolio
weights = pd.DataFrame([0., 57.71, 13.14, 29.16, 0.0]).T
weights.columns = df.columns
weights




<h3 id="Portfolio-return">Portfolio return<a class="anchor-link" href="#Portfolio-return">¶</a></h3>


In [ ]:

# Portfolio returns
port_ret = np.dot(returns, weights.T)
port_mean = port_ret.mean()
port_mean




<h3 id="Portfolio-Volatility">Portfolio Volatility<a class="anchor-link" href="#Portfolio-Volatility">¶</a></h3>


In [ ]:

# Portfolio volatility
port_stdev = np.sqrt(multi_dot([weights, returns.cov(), weights.T]))
port_stdev.flatten()[0]




<h3 id="Portfolio-VaR">Portfolio VaR<a class="anchor-link" href="#Portfolio-VaR">¶</a></h3>


In [ ]:

# Calculate Portfolio VaR at difference confidence level
pVaR_90 = norm.ppf(1-0.90,port_mean,port_stdev).flatten()[0]
pVaR_95 = norm.ppf(1-0.95,port_mean,port_stdev).flatten()[0]
pVaR_99 = norm.ppf(1-0.99,port_mean,port_stdev).flatten()[0]



In [ ]:

# Ouput results in tabular format
ptable = [['90%', pVaR_90],['95%', pVaR_95],['99%', pVaR_99]]
header = ['Confidence Level', 'Value At Risk']
print(tabulate(ptable,headers=header))




<p>Let's now compare the portfolio VaR numbers with that of the individual stocks</p>


In [ ]:

# Iterate over symbols
portpos=0.
for stock in df.columns:
    pos = df[stock].iloc[-1] * 1000*weights[stock][0]
    pvar = VaR(returns)[stock].iloc[1]

    print(f'{stock} Holding Value: {pos:0.8}') 
    print(f'{stock} VaR at 95% confidence level: {pvar:0.8}%')
    print()

    portpos += pos

print(f'Portfolio Holding Value: {portpos:0.8}')
print(f'Portoflio VaR at 95% confidence level: {pVaR_95:0.3}%')




<p>The VaR for the current portfolio of $\$ 1.8$ million is 2.41%, which is much lesser than the individual VaR numbers. This signifies the effect of diversification by selecting different stocks.</p>



<h1 id="References">References<a class="anchor-link" href="#References">¶</a></h1><ul>
<li><p><a href="https://urldefense.com/v3/__https://docs.scipy.org/doc/scipy/reference/__;!!KGvANbslH1YjwA!pp8ABRV5pvlNNVDnmbQpGv8z5Wc4lDFJ3s74EFrfiGXolaNAXRMhuhP5AupxkJs7xpOLJTeFiw$">Scipy</a></p>
</li>
<li><p><a href="https://urldefense.com/v3/__https://github.com/astanin/python-tabulate__;!!KGvANbslH1YjwA!pp8ABRV5pvlNNVDnmbQpGv8z5Wc4lDFJ3s74EFrfiGXolaNAXRMhuhP5AupxkJs7xpPIoOvwlA$">Tabulate</a></p>
</li>
<li><p>Paul Wilmott (2007), Paul Wilmott introduces Quantitative Finance</p>
</li>
</ul>



<hr/>
<p><a href="https://urldefense.com/v3/__http://twitter.com/kannansi__;!!KGvANbslH1YjwA!pp8ABRV5pvlNNVDnmbQpGv8z5Wc4lDFJ3s74EFrfiGXolaNAXRMhuhP5AupxkJs7xpOvShO3hg$">Kannan Singaravelu</a> | <a href="https://urldefense.com/v3/__https://github.com/kannansingaravelu__;!!KGvANbslH1YjwA!pp8ABRV5pvlNNVDnmbQpGv8z5Wc4lDFJ3s74EFrfiGXolaNAXRMhuhP5AupxkJs7xpPquleyhw$">GitHub</a></p>
<p><font size="3">March, 2022</font></p>
